In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
emotion_data = pd.read_csv('/content/drive/MyDrive/fer2013.csv')
labels_ref = ['Anger', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
num_emotions = len(labels_ref)

In [11]:
emotion_data['pixels'].shape

(35887,)

In [12]:
X_train = []
y_train = []
X_test = []
y_test = []
for index, row in emotion_data.iterrows():
    pixels = row['pixels'].split(" ")
    if row['Usage'] == 'Training':
        X_train.append(np.array(pixels))
        y_train.append(row['emotion'])
    elif row['Usage'] == 'PublicTest':
        X_test.append(np.array(pixels))
        y_test.append(row['emotion'])

In [ ]:
X_train = np.array(X_train).astype(np.float)
y_train = np.array(y_train).astype(np.float)
X_test = np.array(X_test).astype(np.float)
y_test = np.array(y_test).astype(np.float)

#normalizing data between oand 1  
X_train -= np.mean(X_train, axis=0)  
X_train /= np.std(X_train, axis=0)  

X_test -= np.mean(X_test, axis=0)  
X_test /= np.std(X_test, axis=0)  

X_train = X_train.reshape(X_train.shape[0], 48, 48, 1)
X_test = X_test.reshape(X_test.shape[0], 48, 48, 1)

y_train= tf.keras.utils.to_categorical(y_train, num_classes=num_emotions)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=num_emotions)

In [ ]:
model = Sequential()  

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(X_train.shape[1:])))  
model.add(Conv2D(64,kernel_size= (3, 3), activation='relu'))  
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))  
model.add(Dropout(0.5))  

#2nd convolution layer  
model.add(Conv2D(64, (3, 3), activation='relu'))  
model.add(Conv2D(64, (3, 3), activation='relu'))  
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))  
model.add(Dropout(0.5))  

#3rd convolution layer  
model.add(Conv2D(128, (3, 3), activation='relu'))  
model.add(Conv2D(128, (3, 3), activation='relu'))  
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))  

model.add(Flatten())  

#fully connected neural networks  
model.add(Dense(1024, activation='relu'))  
model.add(Dropout(0.2))  
model.add(Dense(1024, activation='relu'))  
model.add(Dropout(0.2))  

model.add(Dense(num_emotions, activation='softmax'))  

model.summary()  

In [ ]:
batch_size = 64  
epochs = 30  
#Compliling the model  
model.compile(loss='categorical_crossentropy',  
              optimizer=Adam(0.001),  
              metrics=['accuracy'])  

#Training the model  
model.fit(X_train, y_train,  
          batch_size=batch_size,  
          epochs=epochs,  
          verbose=1,  
          validation_data=(X_test, y_test),  
          shuffle=True)  


#Saving the  model to  use it later on  
fer_json = model.to_json()  
with open("emotion_detection_model.json", "w") as json_file:  
    json_file.write(fer_json)  
model.save_weights("emotion_detection_weights.h5") 

In [ ]:
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array


In [ ]:
# load an sad image from file
image = load_img('img1.jpg', target_size=(48, 48))
# convert the image pixels to a numpy array
image = img_to_array(image)
image = image/255.0
# reshape data for the model
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
image = tf.image.rgb_to_grayscale(image)
prediction_probs = model.predict(image)
max_prob_index = np.argmax(prediction_probs)
emotion_label = labels_ref[max_prob_index]
print(emotion_label)

In [7]:
# load an happy image from file
image = load_img('img2.jpg', target_size=(48, 48))
# convert the image pixels to a numpy array
image = img_to_array(image)
image = image/255.0
# reshape data for the model
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
image = tf.image.rgb_to_grayscale(image)
prediction_probs = model.predict(image)
max_prob_index = np.argmax(prediction_probs)
emotion_label = labels_ref[max_prob_index]
print(emotion_label)

FileNotFoundError: ignored

In [8]:
# load an fear image from file
image = load_img('img3.jpg', target_size=(48, 48))
# convert the image pixels to a numpy array
image = img_to_array(image)
image = image/255.0
# reshape data for the model
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
image = tf.image.rgb_to_grayscale(image)
prediction_probs = model.predict(image)
max_prob_index = np.argmax(prediction_probs)
emotion_label = labels_ref[max_prob_index]
print(emotion_label)

FileNotFoundError: ignored